# Efficient determination of zero-crossings in noisy real-life time series
## Advanced Data Science Capstone Project
### Model evaluation.
In this notebook, the and trained models are evaluated. The presented functions are called at the end of the whole simulation process. 

First, all necessary libraries are imported.



In [ ]:
#Here, the path to the file [Zero_crossings_in_time_series]_import_libraries_python.ipynb should be indicated.
try:
  %run /content/[Zero_crossings_in_time_series]_import_libraries_python.ipynb
except:
  None

     |████████████████████████████████| 204.8MB 72kB/s 
     |████████████████████████████████| 204kB 19.3MB/s 
  Created wheel for pyspark: filename=pyspark-3.0.2-py2.py3-none-any.whl size=205186687 sha256=08411387ff2ccf18a943a0f6b0659c2a3c9116fd3f67e15d5490c424487bdfc5
  Stored in directory: /root/.cache/pip/wheels/8b/09/da/c1f2859bcc86375dc972c5b6af4881b3603269bcc4c9be5d16
Successfully built pyspark


Then, after the simulation, the obtained results are studied. Here, both `t` and `x` are two numpy arrays containing the values of $t$ and $x = g(t)$, respectively, while `first_zero` is the time, when there was a real first zero-crossing. 

First, all zero-crossings in t and x are checked. Then, the one closest to `first_zero` is found. If it is close enough to `first_zero`, then the problem is considered to be solved (i.e., if the accuracy is greater than 99%), otherwise, its considered to be unsolved. The relative error and the approximated first zero-crossing are returned. 

In [ ]:
def evaluate_zero_crossings(t,x,first_zero=None):
  zero_crossings_approx = []
  for i in range(1,len(t)):
    if np.sign(x[i])*np.sign(x[i-1])<=0:
      w0 = 1/(np.finfo(float).eps+abs(x[i-1]))
      w1 = 1/(np.finfo(float).eps+abs(x[i]))
      zero_crossings_approx.append((w1*t[i]+w0*t[i-1])/(w0+w1))
  if first_zero==None:
    try:
      return np.nan,zero_crossings_approx[0],zero_crossings_approx
    except:
      return np.nan,np.nan,zero_crossings_approx
  else:
    if len(zero_crossings_approx)>0:
      first_zero_approx = zero_crossings_approx[np.argmin(np.sqrt((np.array(zero_crossings_approx)-first_zero)**2))]
    else:
      first_zero_approx = -1
    relative_error = abs(first_zero_approx-first_zero)/first_zero
    relative_accuracy_percent = 100*(1-relative_error)
    if (relative_accuracy_percent)>=99:
        print('Solved! :)')
        print('Found zero-crossing at ', first_zero_approx)
        print('Real zero-crossing was at ', first_zero)
        print('Accuracy is ',relative_accuracy_percent,'%')
    else:
        print('Unsolved :(')
        print('Found zero-crossing at ', first_zero_approx)
        print('Real zero-crossing was at ', first_zero)
        print('Accuracy is ',relative_accuracy_percent,'%')
    return relative_error,first_zero_approx,zero_crossings_approx

The presented function can be called both after the simulation or during the simulation. In the latter case, since there cannot be an a priori given first zero-crossing value, it can be omitted (the default value -1 is used and the relative accuracy, error and first_zero_approx are senseless in this case, only the list of zero_crossings_approx is important). This procedure can be used to check if there are zero-crossings in the predicted interval on-the-fly:

In [ ]:
import pandas as pd
t = np.arange(0,1.01,0.1)
x = np.random.normal(0,1,t.shape)
RE,FZA,ZCA = evaluate_zero_crossings(t,x)
display(pd.DataFrame([t,x]).transpose().rename(columns = {0:"t",1:"x"}))
print("Relative error: ",RE)
print("First zero-crossing approximated:", FZA)
print("All approximated zero-crossings:",ZCA)
if len(ZCA)>0:
  print("Warning! A zero-crossing has been determined! The first zero-crossing was determined approximately at t = ",FZA)

,t,x
0,0.0,0.211337
1,0.1,-1.242767
2,0.2,-0.252810
3,0.3,0.369567
4,0.4,0.677532
5,0.5,-0.970068
6,0.6,-0.656965
7,0.7,0.045511
8,0.8,-3.024810
9,0.9,1.896071


Relative error:  nan
First zero-crossing approximated: 0.014533836182406766
All approximated zero-crossings: [0.014533836182406766, 0.24062013755331726, 0.44112234690308033, 0.6935213907927255, 0.7014822780911204, 0.8614688650861801, 0.9382351494493972]
Warning! A zero-crossing has been determined! The first zero-crossing was determined approximately at t =  0.014533836182406766
